In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from envs.CarRacing import CarRacing
from networks.CQNet import CQNetImage
from networks.MuNet  import MuNetImage
# from memory.RewardMemory import Memory
from memory.TorchMemory import Memory
from tqdm.notebook import tqdm
import numpy as np
import sys
import traceback
from networks.utils import *
import torch
import matplotlib.pyplot as plt

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [7]:
# Create environment
env = CarRacing()

# Create network
critic1 = CQNetImage(input_channels=3, hidden_dim=128, action_dim=3).to(device)
critic2 = CQNetImage(input_channels=3, hidden_dim=128, action_dim=3).to(device)
actor = MuNetImage(input_channels=3, hidden_dim=128, action_dim=3).to(device)

critic1_copy = CQNetImage(input_channels=3, hidden_dim=128, action_dim=3).to(device)
critic2_copy = CQNetImage(input_channels=3, hidden_dim=128, action_dim=3).to(device)
actor_copy = MuNetImage(input_channels=3, hidden_dim=128, action_dim=3).to(device)
critic1_copy.copyfrom(critic1)
critic2_copy.copyfrom(critic2)
actor_copy.copyfrom(actor)

# Create replay memory
memory = Memory(3, env.states, 3, 30000)

epsilon = 1.0
epsilon_decay = 0.97
epsilon_min = 0.05

gamma = 0.99
batch_size = 256

sigma = 0.25
sigma = torch.tensor(sigma).to(device)
tau = 0.005  # Soft update parameter

td3_rewards_per_ep = {} # store rewards for each episode

# Update target networks
policy_update_freq = 2  # Delayed policy updates
update_counter = 0

for e in tqdm(range(200)):
    # Reset environment
    state = env.reset()
    ep_reward = 0
    cont = 0
    
    for t in range(500):
        # action = actor(state).cpu().data.numpy()
        # action = np.random.normal(loc=action, scale=sigma).reshape(3)
        action = actor(state)
        noise = torch.normal(mean=torch.zeros_like(action), std=sigma).to(device)
        action = action + noise
        action = action.reshape(3)
        #print(action.reshape(3))
    
        # Step environment
        obs, r, terminal, truncated, info = env.step(2*action, cpu=True)
        ep_reward += r

        #print(obs)
        #print(obs.shape)

        obs_tensor = torch.tensor(obs).to(device)
        r_tensor = torch.tensor(r).to(device)
        terminal_tensor = torch.tensor(terminal).to(device)
        memory.add(obs_tensor[0, 2, :, :], action, r_tensor, terminal_tensor)
        
        # If at least 1000 transitions in memory, sample minibatch and learn
        if len(memory) >= batch_size:
            minibatch = memory.sample(batch_size)
            states, actions, rewards, next_states, terminals = minibatch
            states = states.to(device)
            actions = actions.to(device)
            next_states = next_states.to(device)
            rewards = rewards.to(device)
            terminals = terminals.to(device)

            
            #print(isinstance(states, torch.Tensor))
            # Compute target values for critic networks with added noise for TD3
            next_actions = actor_copy(next_states)
            # Generate noise with the same device as next_actions
            noise = torch.clamp(torch.normal(0, 0.2, size=next_actions.shape, device=next_actions.device), -0.5, 0.5)
            next_actions = torch.clamp(next_actions + noise, env.action_space.low[0], env.action_space.high[0])

            # print(next_states.shape)
            # print(next_actions.shape)
            
            next_q_values1 = critic1_copy(next_states, next_actions, cpu=False)
            next_q_values2 = critic2_copy(next_states, next_actions, cpu=False)
            next_q_values = torch.minimum(next_q_values1, next_q_values2).to(device) 
            targets = rewards + gamma * next_q_values * (1 - terminals)

            # Update critic networks
            critic1.update(states, actions, targets, cpu=False, retain_graph=True)
            critic2.update(states, actions, targets, cpu=False)

            # Update actor and target networks if necessary (delayed)
            update_counter += 1
            if update_counter % policy_update_freq == 0:
                # Update actor network
                actor.update(states, critic1)
                
                # Soft update target networks
                critic1_copy.soft_update(critic1, tau)
                critic2_copy.soft_update(critic2, tau)
                actor_copy.soft_update(actor, tau)

        state = obs

        if terminal or truncated:
            break
        
        cont += 1
    
    td3_rewards_per_ep[e] = [ep_reward, cont]

    if epsilon > epsilon_min:
        epsilon *= epsilon_decay

# Close environment
#env.plotnetwork(actor, critic1)
env.close()

  0%|          | 0/200 [00:00<?, ?it/s]

Traceback (most recent call last):
  File "c:\Users\jpkqv\anaconda3\envs\rlga\lib\site-packages\debugpy\_vendored\pydevd\_pydevd_bundle\pydevd_vars.py", line 478, in change_attr_expression
    value = eval(expression, frame.f_globals, frame.f_locals)
  File "<string>", line 1, in <module>
NameError: name 'array' is not defined
Traceback (most recent call last):
  File "_pydevd_bundle/pydevd_cython.pyx", line 1078, in _pydevd_bundle.pydevd_cython.PyDBFrame.trace_dispatch
  File "_pydevd_bundle/pydevd_cython.pyx", line 297, in _pydevd_bundle.pydevd_cython.PyDBFrame.do_wait_suspend
  File "c:\Users\jpkqv\anaconda3\envs\rlga\lib\site-packages\debugpy\_vendored\pydevd\pydevd.py", line 1976, in do_wait_suspend
    keep_suspended = self._do_wait_suspend(thread, frame, event, arg, suspend_type, from_this_thread, frames_tracker)
  File "c:\Users\jpkqv\anaconda3\envs\rlga\lib\site-packages\debugpy\_vendored\pydevd\pydevd.py", line 2011, in _do_wait_suspend
    time.sleep(0.01)
KeyboardInterrupt


KeyboardInterrupt: 